In [1]:
# !pip install biom-format

In [54]:
import biom
import torch
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np

In [133]:
# Load a BIOM file
table = biom.load_table("fecal.biom")
s_ids = table.ids()
obs_ids = table.ids(axis='observation')

# # Explore the table
# print("Shape:", table.shape, "\n")  # Get the number of observations and samples
# print("Observation ids:", table.ids(axis='observation'), "\n")  # Get observation IDs
# print("Sample ids:", table.ids(axis='sample'), "\n")  # Get sample IDs
# print("Observation meta data:", table.metadata(axis='observation'), "\n")  # Get observation metadata
# print("Sample meta data:", table.metadata(axis='sample'), "\n")  # Get sample metadata

# # Access data
# print(table.matrix_data)  # Access the raw data matrix

# Convert to other formats
df = table.to_dataframe()  # Convert to pandas DataFrame
df

,14812.CV.19.R077,14812.CV.19.R071,14812.CV.19.009,14812.CV.19.079,14812.CV.19.046,14812.CV.19.R024,14812.CV.19.043,14812.CV.19.080,14812.CV.19.R069,14812.CV.19.R078,...,14812.N.070,14812.N.014,14812.N.060,14812.N.073,14812.N.062,14812.N.037,14812.N.051,14812.N.032,14812.N.072,14812.N.016
TACAGAGGTCTCAAGCGTTGTTCGGAATCACTGGGCGTAAAGCGTGCGTAGGCTGTTTCGTAAGTCGTGTGTGAAAGGCGCGGGCTCAACCCGCGGACGGCACATGATACTGCGAGACTAGAGTAATGGAGGGGGAACCGGAATTCTCGG,969.0,0,0,0,0,0,0,0,0,0,...,550.0,68.0,0,0,0,0,0,0,0,2909.0
TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGGCGGACGCTTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTGCAGTTGATACTGGGTGTCTTGAGTACAGTAGAGGCAGGCGGAATTCGTGG,858.0,0,521.0,1.0,171.0,32.0,107.0,249.0,3211.0,20.0,...,6.0,55.0,2286.0,824.0,2509.0,14.0,2306.0,1930.0,2516.0,0
TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGATGGATGTTTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTGCAGTTGATACTGGATATCTTGAGTGCAGTTGAGGCAGGCGGAATTCGTGG,430.0,184.0,530.0,5.0,739.0,192.0,308.0,413.0,251.0,112.0,...,7.0,160.0,0,1380.0,0,8.0,635.0,2688.0,3443.0,1696.0
TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGACGGCACGGCAAGCCAGATGTGAAAGCCCGGGGCTCAACCCCGGGACTGCATTTGGAACTGCTGAGCTAGAGTGTCGGAGAGGCAAGTGGAATTCCTAG,232.0,33.0,0,7.0,118.0,509.0,17.0,511.0,12.0,55.0,...,153.0,336.0,94.0,136.0,0,231.0,106.0,280.0,35.0,149.0
TACGTAGGTGGCGAGCGTTGTCCGGAATTATTGGGCGTAAAGAGCATGTAGGCGGCTTAATAAGTCGAGCGTGAAAATGCGGGGCTCAACCCCGTATGGCGCTGGAAACTGTTAGGCTTGAGTGCAGGAGAGGAAAGGGGAATTCCCAGT,200.0,0,0,0,0,16.0,0,333.0,73.0,49.0,...,0,31.0,0,945.0,0,0,0,1147.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TACGTAGGTGGCAAGCGTTGTCCGGATTTACTGGGTGTAAAGGGCGTGTAGGCGGGTAGACAAGTCAGATGTGAAATACCGGGGCTCAACTCCGGGGCTGCATTTGAAACTGTATATTTTGAGTGTCGGAGAGGAAAGCGGAATTCCTAG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
TACGTATGGTGCAAGCGTTATCCGGATTTACTGGGCGTAAAGGGAGCGCAGGCGGTGCGGCAAGTCTGATGTGAAAGCCCGGGGCTCAACCCCGGTACTGCATTTGAAACTGTCTAGCTAGAGTGCAGGAGAGGTAAGCGGAATTCCTAG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
TACGTAGGGAGCGAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGTAGGCGGGATGGCAAGTTGGATGTTTAAACTAACGGCTCAACTGTTAGGTGCATCCAAAACTGCTGTTCTTGAGTGAAGTAGAGGCAGGCGGGATTCCTAGT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
TACGTAGGTGGCGAGCGTTGTCCGGATTTACTGGGCGTAAAGGGTGCGTAGGCGGCTTGATAAGTTAGAGGTGAAATACCGGTGCTTAACACCGGAACTGCCTCTAATACTGTTGAACTAGAGAGTAGTTGCGGTAGGCGGAATGTATGG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0


In [4]:
#S is the sample id
#R is the observation id
#C is the number of times the sequence was observed in the sample
#coo.data is the counts

#coo.row and coo.col are x and y coordinates respectively
coo = table.matrix_data.tocoo()
counts, rows, cols = coo.data, coo.row, coo.col

In [16]:
coo = table.matrix_data.tocoo()
counts, rows, cols = coo.data, coo.row, coo.col

src = pd.DataFrame({
    'sample': [s_ids[col] for col in cols],
    'asv': [obs_ids[row] for row in rows],
    'count': coo.data
})
src.head()

,sample,asv,count
0,14812.CV.19.R077,TACAGAGGTCTCAAGCGTTGTTCGGAATCACTGGGCGTAAAGCGTG...,969.0
1,14812.CV.19.041,TACAGAGGTCTCAAGCGTTGTTCGGAATCACTGGGCGTAAAGCGTG...,2.0
2,14812.CV.19.047,TACAGAGGTCTCAAGCGTTGTTCGGAATCACTGGGCGTAAAGCGTG...,16.0
3,14812.CV.19.010,TACAGAGGTCTCAAGCGTTGTTCGGAATCACTGGGCGTAAAGCGTG...,16.0
4,14812.CV.19.086,TACAGAGGTCTCAAGCGTTGTTCGGAATCACTGGGCGTAAAGCGTG...,11.0


In [17]:
src.groupby('sample').count().sort_values('asv', ascending=False).head()

,asv,count
sample,,
14812.CV.19.R083,562,562
14812.CV.19.041,514,514
14812.CV.19.043,505,505
14812.N.016,485,485
14812.CV.19.048,484,484


In [18]:
print(
    f'input dimensions: \
    {len(s_ids)} \
    x {src.groupby("sample").count().get("asv").max()} \
    x {src.get("asv").apply(len).max()} \
    x {152}'
)

input dimensions:     60     x 562     x 150     x 152


In [19]:
# embedding dimension
nuc_vocab = {'A': 0, 'T': 1, 'C': 2, 'G': 3}
embed_dim = 152
nuc_embed = torch.nn.Embedding(num_embeddings=len(nuc_vocab), embedding_dim=embed_dim)
nuc_embed

Embedding(4, 152)

In [20]:
def encode_seq(seq):
    nuc_vocab = {'A': 0, 'T': 1, 'C': 2, 'G': 3}
    embed_dim = 152

    # get tensor of encoded nucleotides
    seq_indices = torch.tensor([nuc_vocab[nuc] for nuc in seq])

    # get embedding
    nuc_embed = torch.nn.Embedding(num_embeddings=len(nuc_vocab), embedding_dim=embed_dim)
    seq_embeddings = nuc_embed(seq_indices)
    
    return seq_embeddings

In [21]:
coo = table.matrix_data.tocoo()
counts, rows, cols = coo.data, coo.row, coo.col
    
# dictionary key: sample, value: encoded asv
asv_data = {s_id: [] for s_id in s_ids}
for count, row, col in zip(counts, rows, cols):
    s_id = table.ids()[col]
    obs_seq = table.ids(axis='observation')[row]
    asv_data[s_id].append(encode_seq(obs_seq))

In [22]:
# for 1 sample: # asvs X x nucleotides x embedding
torch.stack(asv_data[s_id]).size()

torch.Size([485, 150, 152])

In [23]:
# Batch x encoded ASVs x Nucleotides x Embedding
samples = {key: torch.stack(value) for key, value in asv_data.items()}
model_input = pad_sequence(samples.values(), batch_first=True)
model_input.size()

torch.Size([60, 562, 150, 152])

In [48]:
coo = table.matrix_data.tocoo()
counts, rows, cols = coo.data, coo.row, coo.col
    
# dictionary key: sample, value: encoded asv
asv_data_raw = {s_id: [] for s_id in s_ids}
for count, row, col in zip(counts, rows, cols):
    s_id = table.ids()[col]
    obs_seq = table.ids(axis='observation')[row]
    asv_data_raw[s_id].append(obs_seq)

In [104]:
model_input_raw = [np.array(value) for key, value in asv_data_raw.items()]

# pad asv
max_len = max(len(arr) for arr in model_input_raw)
model_input_raw = [np.pad(arr, (0, max_len - len(arr)), mode='constant', constant_values='N') for arr in model_input_raw]

# Convert to array
model_input_raw = np.array(model_input_raw)
model_input_raw.shape

(60, 562)

In [130]:
asv_data_raw['14812.CV.19.R024'][2] == raw_input_df.loc['14812.CV.19.R024'][2]

True

In [136]:
print(biom.__version__)

2.1.16


In [128]:
# row: sample
# col: asv
raw_input_df = pd.DataFrame(model_input_raw)
raw_input_df['sample'] = asv_data_raw.keys()
raw_input_df = raw_input_df.set_index('sample')
raw_input_df.to_csv('fecal_by_sample.csv')

In [129]:
raw_input_df

,0,1,2,3,4,5,6,7,8,9,...,552,553,554,555,556,557,558,559,560,561
sample,,,,,,,,,,,,,,,,,,,,,
14812.CV.19.R077,TACAGAGGTCTCAAGCGTTGTTCGGAATCACTGGGCGTAAAGCGTG...,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGTGGCGAGCGTTGTCCGGAATTATTGGGCGTAAAGAGCA...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTATGGAGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGTG...,TACGGAGGATTCAAGCGTTATCCGGATTTATTGGGTTTAAAGGGTG...,TACGTAGGGGGCAAGCGTTATCCGGAATTACTGGGTGTAAAGGGTG...,TACGTATGGTGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,...,N,N,N,N,N,N,N,N,N,N
14812.CV.19.R071,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTATGGAGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGTG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTATGGTGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,AACGTAGGGTGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,...,N,N,N,N,N,N,N,N,N,N
14812.CV.19.009,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGGAGGATTCAAGCGTTATCCGGATTTATTGGGTTTAAAGGGTG...,TACGTATGGTGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTATGGTGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,AACGTAGGGTGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAG...,...,N,N,N,N,N,N,N,N,N,N
14812.CV.19.079,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGAATTACTGGGTGTAAAGGGTG...,TACGTATGGTGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTATGGTGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,...,N,N,N,N,N,N,N,N,N,N
14812.CV.19.046,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTATGGAGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGTG...,TACGTAGGGGGCAAGCGTTATCCGGAATTACTGGGTGTAAAGGGTG...,TACGTATGGTGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,...,N,N,N,N,N,N,N,N,N,N
14812.CV.19.R024,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGTGGCGAGCGTTGTCCGGAATTATTGGGCGTAAAGAGCA...,TACGTAGGGGGCAAGCGTTATCCGGAATTACTGGGTGTAAAGGGTG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTATGGTGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,AACGTAGGGTGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAG...,...,N,N,N,N,N,N,N,N,N,N
14812.CV.19.043,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTATGGAGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGTG...,TACGGAGGATTCAAGCGTTATCCGGATTTATTGGGTTTAAAGGGTG...,TACGTAGGGGGCAAGCGTTATCCGGAATTACTGGGTGTAAAGGGTG...,TACGTATGGTGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,...,N,N,N,N,N,N,N,N,N,N
14812.CV.19.080,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,TACGTAGG

In [ ]:
pd.read_csv()

In [ ]:
# Batch x ASVs raw x Nucleotides x Embedding
samples = {key: np.array(value) for key, value in asv_data_raw.items()}
model_input_raw = pad_sequence(samples.values(), batch_first=True)
model_input_raw.size()

In [24]:
file_path = "sample_information_from_prep_16151.tsv"
sample_info = pd.read_csv(file_path, sep="\t")[['sample_name','covid_positive']]

sample_info[['sample_name','covid_positive']].head()

,sample_name,covid_positive
0,14812.N.066,no
1,14812.N.051,no
2,14812.N.068,no
3,14812.N.037,no
4,14812.CV10.R025,recovered


In [41]:
fecal = src.merge(sample_info, left_on='sample', right_on='sample_name').drop(columns=['sample_name'])
fecal.to_csv('fecal.csv', index=False)